In [1]:
import os, glob, re
import pandas as pd

RAW_DIR = "raw/brfss_year/"
OUT_DIR = "processed/"
os.makedirs(OUT_DIR, exist_ok=True)

crosswalk = pd.read_csv("county_fips_crosswalk.csv", dtype=str)

# Collect yearly files
all_files = glob.glob(os.path.join(RAW_DIR, "brfss_*.csv"))
csv_files = [f for f in all_files if re.search(r"brfss_\d{4}\.csv$", f)]
csv_files = sorted(csv_files)

print(f"Found {len(csv_files)} yearly files")

Found 0 yearly files


In [373]:
print("OUT_DIR absolute path:", os.path.abspath(RAW_DIR))
print("OUT_DIR absolute path:", os.path.abspath(OUT_DIR))

OUT_DIR absolute path: /Users/umichmads/brfss_laus_week2/BRFSS/all_years/raw/brfss_year
OUT_DIR absolute path: /Users/umichmads/brfss_laus_week2/BRFSS/all_years/processed


In [764]:
# --- Choose which year to look at ---
TEST_YEAR = "2020"   # change this to e.g. "2000", "2005" etc.

# Find the matching file
year_file = [f for f in csv_files if TEST_YEAR in f]
if not year_file:
    raise ValueError(f"No file found for year {TEST_YEAR}")
fpath = year_file[0]

print(f"\n🔄 Processing year {TEST_YEAR}: {fpath}")

# Load the data
brfss = pd.read_csv(fpath, dtype=str)

# Show the head
print(f"\n[{TEST_YEAR}] Preview of raw BRFSS file:")
display(brfss.head())


🔄 Processing year 2020: raw/brfss_year/brfss_2020.csv

[2020] Preview of raw BRFSS file:


,_state,_geostr,_denstr2,precall,repnum,repdepth,fmonth,idate,imonth,iday,...,_rfbing5,_drnkdy4,_drnkmo4,_rfdrhv4,_rfdrmn4,_rfdrwm4,_aidtst3,havhpad,year,state_fips
0,1.0,1.0,1.0,1.0,10001.0,27.0,1.0,b'01202011',b'01',b'20',...,1.0,86.0,26.0,1.0,NaN,1.0,2.0,NaN,2020,01
1,1.0,1.0,1.0,1.0,10008.0,13.0,1.0,b'01142011',b'01',b'14',...,9.0,9900.0,9999.0,9.0,9.0,NaN,NaN,NaN,2020,01
2,1.0,5.0,1.0,1.0,10058.0,4.0,1.0,b'01062011',b'01',b'06',...,1.0,14.0,4.0,1.0,NaN,1.0,2.0,NaN,2020,01
3,1.0,7.0,1.0,1.0,10079.0,7.0,1.0,b'02012011',b'02',b'01',...,9.0,9900.0,9999.0,9.0,NaN,9.0,NaN,NaN,2020,01
4,1.0,7.0,1.0,1.0,10081.0,28.0,1.0,b'02012011',b'02',b'01',...,1.0,5.397605346934028e-79,5.397605346934028e-79,1.0,NaN,1.0,1.0,NaN,2020,01


In [765]:
print("BRFSS shape:", brfss.shape)
print("Crosswalk shape:", crosswalk.shape)

BRFSS shape: (506467, 456)
Crosswalk shape: (3220, 3)


In [766]:
# --- 2. Try County-Level FIPS ---

# Extract only digits, squeeze to Series, pad
brfss["_state"] = (
    brfss["_state"].astype(str).str.extract(r'(\d+)').iloc[:, 0].fillna("").str.zfill(2)
)
brfss["ctycode"] = (
    brfss["ctycode"].astype(str).str.extract(r'(\d+)').iloc[:, 0].fillna("").str.zfill(3)
)

# Build FIPS
brfss["fips"] = brfss["_state"] + brfss["ctycode"]

# ✅ Filter to valid county rows only (exclude 000 placeholder counties)
brfss = brfss[brfss["ctycode"] != "000"].copy()

print("Remaining rows with valid county codes:", len(brfss))
print("Sample FIPS after filter:", brfss["fips"].unique()[:10])

KeyError: 'ctycode'

In [ ]:
# --- 3. Merge Geography ---
print("✅ Using county-level FIPS for merge")
brfss = brfss.merge(crosswalk, on="fips", how="left")
geo_cols = ["fips", "state_name", "county_name"]

print("Rows after merge:", len(brfss))
print("Sample with names:", brfss[geo_cols].head())

In [ ]:
# Check a few rows of _state, ctycode, and fips
print(brfss[["_state", "ctycode", "fips"]].head(20))

# How many distinct state codes vs county codes
print("Unique _state codes:", brfss["_state"].nunique())
print("Unique ctycode codes:", brfss["ctycode"].nunique())

# Count how many fips are just padding (state code + '000')
print("FIPS ending in '000' (likely missing county):",
      (brfss["ctycode"] == "000").sum())

# Compare to crosswalk coverage
print("Sample BRFSS FIPS:", brfss["fips"].dropna().unique()[:10])
print("Sample Crosswalk FIPS:", crosswalk["fips"].dropna().unique()[:10])

# ✅ After merge, how many matched?
matched = brfss["county_name"].notna().sum()
total = len(brfss)
print(f"Matched county names: {matched}/{total} ({matched/total:.1%})")

In [ ]:
print(brfss.columns.tolist())

In [ ]:
job_a_map = {
    # --- Survey design & metadata ---
    "_state": "state_fips_code",
    "_geostr": "geographic_stratum",
    "_denstr": "density_stratum",
    "_psu": "primary_sampling_unit",
    "_record": "record_type",
    "idate": "interview_date",
    "imonth": "interview_month",
    "iday": "interview_day",
    "iyear": "interview_year",
    "intvid": "interviewer_id",
    "dispcode": "disposition_code",
    "winddown": "winddown_flag",
    "seqno": "sequence_number",
    "nattmpts": "attempts_at_contact",
    "nrecsel": "record_selection_number",
    "nrecstr": "record_stratum_number",
    "bpselwt": "bp_selection_weight",
    "wakss1s2": "wakss_stage_variable",

    # --- Household composition ---
    "numadult": "num_adults_in_household",
    "nummen": "num_men_in_household",
    "numwomen": "num_women_in_household",

    # --- General health ---
    "genhlth": "general_health_status",
    "physhlth": "days_physical_health_not_good",
    "menthlth": "days_mental_health_not_good",
    "poorhlth": "days_poor_health_limited_activities",

    # --- Insurance / coverage ---
    "hlthplan": "has_any_health_plan",
    "medicar2": "covered_by_medicare",
    "typcovr1": "coverage_type_primary",
    "typcovr2": "coverage_type_secondary",
    "nocov12": "no_health_coverage_12mo",
    "pastplan": "had_previous_plan",
    "medcost": "could_not_see_doctor_due_to_cost",
    "checkup": "time_since_last_checkup",

    # --- Blood pressure / cholesterol ---
    "bptake": "blood_pressure_taken",
    "bphigh": "ever_told_high_bp",
    "highgt1": "told_high_bp_more_than_once",
    "bloodcho": "ever_had_cholesterol_checked",
    "cholchk": "cholesterol_checked_5yr",
    "toldhi": "ever_told_high_cholesterol",
    "diabetes": "ever_told_diabetes",

    # --- Dental / oral health ---
    "lastden2": "last_dentist_visit",
    "rmvteeth": "teeth_removed_due_to_decay",
    "denclean": "teeth_cleaned_by_dentist",

    # --- Skin cancer / sun ---
    "sunburn": "sunburn_past_year",
    "numburn": "num_sunburns_past_year",

    # --- Tobacco (beginning) ---
    "smoke100": "smoked_100_cigarettes",
    "smokeday": "current_smoking_frequency",
    "smokenum": "cigs_smoked_per_day",
    "smoknm30": "cigs_smoked_past_30d",
    "stopsmok": "ever_attempted_quit_smoking",
    "lastsmok": "time_since_last_smoked",

    # --- Alcohol use ---
    "drinkany": "any_alcohol_past_month",
    "alcohol": "avg_drinks_per_week",
    "nalcocc": "num_alcoholic_occasion",
    "drinkge5": "binge_drinking_occurrence",
    "drinkdri": "drank_and_drove",

    # --- Core demographics ---
    "age": "respondent_age",
    "orace": "respondent_race",
    "hispanic": "hispanic_ethnicity",
    "marital": "marital_status",
    "chld04": "children_under_5_in_household",
    "chld0512": "children_5_12_in_household",
    "chld1317": "children_13_17_in_household",
    "educa": "education_level",
    "employ": "employment_status",
    "income2": "household_income_category",
    "weight": "respondent_weight_pounds",
    "height": "respondent_height_inches",
    "htf": "height_feet",
    "hti": "height_inches_remainder",
    "ctycode": "county_code",
    "numhhold": "num_households",
    "numphons": "num_phones",
    "sex": "respondent_sex",
}

In [ ]:
job_b_map = {
    # --- Cancer screenings & reproductive health ---
    "hadmam": "ever_had_mammogram",
    "howlong": "time_since_last_mammogram",
    "whydone": "reason_for_last_mammogram",
    "profexam": "ever_had_clinical_breast_exam",
    "lengexam": "time_since_last_clinical_breast_exam",
    "reasexam": "reason_for_last_clinical_breast_exam",
    "hadpap": "ever_had_pap_smear",
    "lastpap": "time_since_last_pap_smear",
    "whypap": "reason_for_last_pap_smear",
    "hadhyst": "ever_had_hysterectomy",
    "pregnant": "currently_pregnant",

    # --- Vaccinations ---
    "flushot": "flu_shot_past_year",
    "fluprov": "where_received_flu_shot",
    "pneumvac": "ever_had_pneumonia_vaccine",

    # --- Cancer screening (colorectal) ---
    "bldstool": "ever_had_blood_stool_test",
    "lstbldst": "time_since_last_blood_stool_test",
    "hadsigm": "ever_had_sigmoidoscopy_colonoscopy",
    "lastsigm": "time_since_last_sigmoidoscopy",

    # --- Reproductive / household children ---
    "chldlt16": "children_lt16_in_household",

    # --- Safety / helmets ---
    "bikehlmt": "wears_bike_helmet",

    # --- Smoking detection proxies ---
    "smkdete2": "smoke_detector_present",

    # --- Education proxy ---
    "grade2": "highest_grade_completed",

    # --- HIV / STD testing & risk ---
    "conduse": "condom_use",
    "gethiv": "ever_tested_hiv",
    "donbld85": "donated_blood_since_1985",
    "don12mo": "donated_blood_past_12mo",
    "hivtst2a": "ever_tested_hiv2",
    "hivtst2b": "tested_hiv_in_past_12mo",
    "hiv12mo": "tested_hiv_last_12mo",
    "test12mo": "other_test_past_12mo",
    "rsntst2": "reason_not_tested_hiv",
    "whrtst3": "where_tested_hiv",
    "results": "received_hiv_test_results",
    "counsel": "received_hiv_counseling",

    # --- Diabetes management ---
    "diabage": "age_told_diabetes",
    "insulin": "currently_use_insulin",
    "inslnfrq": "insulin_frequency",
    "bldsugar": "blood_sugar_test_frequency",
    "hemoglbn": "ever_tested_hemoglobin_a1c",
    "doctdiab": "doctor_visits_for_diabetes",
    "chkhemo": "hemoglobin_a1c_check_frequency",
    "feetchk": "feet_check_frequency",
    "eyeexam": "last_eye_exam",
    "visndist": "trouble_seeing_distance",
    "visnread": "trouble_reading_print",
    "visiontv": "trouble_seeing_tv",

    # --- Sexual behavior / risk ---
    "sexintmn": "sex_partners_past_months",
    "sexcondm": "condom_use_last_intercourse",
    "condlast": "time_since_last_condom_use",
    "coneff2": "condom_effectiveness_perception",
    "newpartn": "num_new_sex_partners",
    "hivrisk": "perceived_hiv_risk",
    "stdtreat": "treated_for_std",
    "stdclin": "visited_std_clinic",
    "sexbeha2": "high_risk_sex_behavior",
    "selcptn3": "contraception_selection",
    "sex1ptn3": "num_sex_partners_lifetime",
    "usecond3": "uses_condom_regularly",

    # --- Pregnancy history ---
    "preglst5": "pregnancies_last_5yrs",
    "lstpreg": "time_since_last_pregnancy",
    "lstpreg2": "age_last_pregnancy",

    # --- Birth control ---
    "brthcntl": "currently_using_birth_control",
    "typcntrl": "type_of_birth_control",
    "rsnnobc": "reason_not_using_birth_control",

    # --- Family planning & services ---
    "whrsvcs": "where_family_planning_services",
    "famplan": "ever_used_family_planning",
    "lastused": "time_since_last_birth_control_use",
    "rsnnocov": "reason_no_health_coverage",
    "mcrelngt": "length_time_on_medicare",
    "doctlist": "doctor_provided_list",
    "certdoct": "certified_doctor_visit",
    "hlthcovr": "other_health_coverage",
    "rsnwocov": "reason_without_coverage",

    # --- Care quality ---
    "ratecare": "rate_healthcare_quality",
    "primcare": "has_primary_care_provider",
    "noprimcr": "reason_no_primary_care",
    "mostcare": "where_usually_seek_care",
    "facilit2": "type_of_health_facility",
    "distcare": "distance_to_care_facility",
    "numbdoc2": "num_doctors_seen",
    "chngdoct": "changed_doctor_recently",
    "lastchng": "time_since_doctor_change",

    # --- Chronic conditions ---
    "asthma": "ever_told_asthma",
    "asthnow": "currently_has_asthma",
    "pcseathb": "provider_counseled_eating_habits",
    "pcsexer": "provider_counseled_exercise",
    "pcsinjpv": "provider_counseled_injury_prevention",
    "pcsdrgab": "provider_counseled_drug_abuse",
    "pcsalch": "provider_counseled_alcohol",
    "pcssmok": "provider_counseled_smoking",
    "pcsaids": "provider_counseled_hiv_aids",
    "cvdfatrk": "cvd_risk_factors_tracked",
    "cvdexrsk": "cvd_exercise_risk",
    "cvdfat02": "cvd_fat_risk",
    "cvdexr02": "cvd_exercise_risk2",
    "cvdinfar": "ever_told_heart_attack",
    "cvdcorhd": "ever_told_coronary_heart_disease",
    "cvdstrok": "ever_told_stroke",
    "cvdasprn": "takes_daily_aspirin",
    "aspunsaf": "advised_aspirin_unsafe",
    "whyaspan": "reason_takes_aspirin",
    "whyaspha": "reason_take_aspirin_heart",
    "whyaspsk": "reason_skip_aspirin",
    "pastmeno": "post_menopausal",
    "dicsestr": "discussed_estrogen_use",
    "estrpill": "ever_used_estrogen_pills",
    "curestro": "estrogen_use_for_condition",
    "estrhart": "estrogen_use_for_heart",
    "estrbone": "estrogen_use_for_bone",
    "estrflsh": "estrogen_use_for_hot_flashes",
    "pain12mn": "chronic_pain_12mo",
    "symtmmth": "symptoms_past_month",
    "lmtjoint": "limited_by_joint_symptoms",
    "havarth": "ever_told_arthritis",
    "typearth": "type_of_arthritis",
    "trtarth": "treatment_for_arthritis",
}

In [ ]:
job_c_map = {
    # --- Nutrition ---
    "fruitjui": "drinks_fruit_juice",
    "fruit": "eats_fruit",
    "greensal": "eats_green_salad",
    "potatoes": "eats_potatoes",
    "carrots": "eats_carrots",
    "vegetabl": "eats_other_vegetables",

    # --- Physical activity ---
    "exerany": "any_physical_activity",
    "exeract1": "primary_exercise_activity",
    "exeroft1": "frequency_primary_exercise",
    "exerhmm1": "duration_primary_exercise_minutes",
    "exeroth2": "did_other_exercise",
    "exeract2": "secondary_exercise_activity",
    "exeroft2": "frequency_secondary_exercise",
    "exerhmm2": "duration_secondary_exercise_minutes",
    "exerdis1": "distance_primary_exercise",
    "exerdis2": "distance_secondary_exercise",
    "phyact": "general_physical_activity_level",

    # --- Weight management ---
    "losewt": "trying_to_lose_weight",
    "maintain": "trying_to_maintain_weight",
    "fewcal": "trying_to_reduce_calories",
    "wtdesire": "desired_weight_pounds",

    # --- Provider advice ---
    "dradvice": "received_weight_advice_from_provider",

    # --- Supplements / vitamins ---
    "vitamins": "takes_vitamins",
    "multivit": "takes_multivitamins",
    "folicacd": "takes_folic_acid",
    "takevit": "times_per_week_takes_vitamins",
    "recommen": "provider_recommended_vitamins",

    # --- Sun protection behaviors ---
    "sunblock": "uses_sunblock",
    "spfnum": "sunblock_spf_number",
    "avoidsun": "avoids_sun_exposure",
    "sunhat": "wears_sun_hat",
    "sunwear": "wears_protective_clothing",
    "sunsens": "skin_sensitivity_to_sun",
    "burntype": "typical_sunburn_severity",
    "howsafe": "perception_of_sun_safety",

    # --- Housing & SES-adjacent ---
    "renthome": "housing_tenure_rent_vs_own",
    "curadrs": "current_address_status",
    "nmclsfrd": "num_close_friends",
    "enghfood": "food_security",

    # --- Tobacco environment ---
    "tobacco": "tobacco_in_home",
    "indoors": "smoking_allowed_indoors",
    "smkpublc": "smoking_allowed_public_places",
    "smkwork": "smoking_allowed_workplace",
    "smkrest": "smoking_allowed_restaurants",
    "smkschls": "smoking_allowed_schools",
    "smkdaycr": "smoking_allowed_daycare",
    "smkindor": "smoking_allowed_indoor_places",

    # --- Smokeless tobacco & current use ---
    "useever": "ever_used_smokeless_tobacco",
    "usenow": "currently_uses_smokeless_tobacco",

    # --- Disability ---
    "disablty": "any_activity_limiting_disability",
}

In [ ]:
job_d_map = {
    # --- Derived anthropometrics ---
    "_bmi": "body_mass_index",
    "_wtforht": "weight_for_height",
    "weight": "respondent_weight_pounds",   # already in Job A but keep consistent
    "height": "respondent_height_inches",
    "htf": "height_feet",
    "hti": "height_inches_remainder",

    # --- Smoking recodes ---
    "_smoker2": "smoking_status_recode",
    "_smkless": "smokeless_tobacco_use_recode",
    "_rfsmok2": "current_smoker_flag",
    "_rftobac": "tobacco_use_flag",

    # --- Drinking recodes ---
    "_drnkmo": "monthly_drinks_estimate",
    "_rfdracu": "acute_drinking_risk_flag",
    "_rfdrdri": "drinking_and_driving_flag",
    "_rfdrchr": "chronic_drinking_risk_flag",

    # --- Fruit & vegetable indices ---
    "_frtserv": "daily_fruit_veg_servings",
    "_frtindx": "fruit_veg_index",
    "_rfregul": "regular_exercise_flag",
    "_rflifes": "lifestyle_activity_flag",

    # --- Hypertension & cholesterol recodes ---
    "_rfhype2": "high_blood_pressure_flag",
    "_rfchol": "high_cholesterol_flag" if "_rfchol" in ["_cholchk"] else None,
    "_cholchk": "cholesterol_screening_flag",

    # --- Obesity flags ---
    "_rfobese": "obesity_flag",
    "_rfwhbmi": "overweight_or_obese_flag",

    # --- Demographic recodes ---
    "_ageg": "age_category",
    "_ageg5yr": "age_group_5yr",
    "_age65yr": "age65plus_flag",
    "_raceg": "race_group",
    "_racegr": "race_group_recode",
    "_raceg2_": "race_group_alt",
    "_sexg_": "sex_group",
    "_impage": "imputed_age",
    "race": "self_reported_race",

    # --- Region & geography ---
    "_region": "census_region",
    "_geowt": "geographic_weight",
    "_msacode": "msa_code",
    "_ststr": "stratum_code",
    "_denwt": "density_weight",
    "_geostr": "geographic_stratum",  # already in Job A but consistent
    "state_fips": "state_fips_duplicate",  # alt coding
    "fips": "county_fips_code",
    "state_name": "state_name",
    "county_name": "county_name",

    # --- Final weights ---
    "_raw": "raw_weight",
    "_csa": "csa_weight",
    "_wt1": "interim_weight",
    "_poststr": "post_stratification_weight",
    "_finalwt": "final_weight",  # main BRFSS weight
    "year": "survey_year",
    "_qstver": "questionnaire_version",
    "_totindx": "total_index_score",
}

In [ ]:
# Combine all jobs into a single renaming dictionary
column_map = {}
column_map.update(job_a_map)
column_map.update(job_b_map)
column_map.update(job_c_map)
column_map.update(job_d_map)

# Apply to dataset
brfss = brfss.rename(columns=column_map)

In [ ]:
print(brfss.tail())

In [ ]:
# --- Geography keys (always included for LAUS crosswalk) ---
geo_cols = ["county_name", "state_name", "survey_year"]

# --- Core Socioeconomic variables (no geo) ---
socio_vars_core = [
    "respondent_age", "respondent_sex", "self_reported_race", "hispanic_ethnicity",
    "marital_status", "education_level", "employment_status", "household_income_category",
    "num_adults_in_household", "num_men_in_household", "num_women_in_household",
    "housing_tenure_rent_vs_own", "food_security",
    "has_any_health_plan", "covered_by_medicare", "could_not_see_doctor_due_to_cost"
]

# --- Core Health Outcomes (no geo) ---
health_vars_core = [
    "general_health_status", "days_physical_health_not_good", "days_mental_health_not_good",
    "days_poor_health_limited_activities",
    "ever_told_high_bp", "ever_told_diabetes",
    "smoked_100_cigarettes", "current_smoking_frequency",
    "any_alcohol_past_month", "avg_drinks_per_week",
    "body_mass_index", "respondent_weight_pounds", "respondent_height_inches",
    "eats_fruit", "eats_other_vegetables", "any_physical_activity"
]

# --- Expanded (add to the core vars only) ---
socio_vars_expanded = socio_vars_core + [
    "county_code", "num_households", "num_phones",
    "ever_used_family_planning", "length_time_on_medicare",
    "coverage_type_primary", "coverage_type_secondary", "no_health_coverage_12mo",
    "had_previous_plan", "reason_without_coverage",
    "age_category", "age_group_5yr", "age65plus_flag"
]

health_vars_expanded = health_vars_core + [
    "cholesterol_checked_5yr", "ever_told_high_cholesterol",
    "ever_told_asthma", "currently_has_asthma",
    "ever_told_heart_attack", "ever_told_coronary_heart_disease",
    "ever_told_stroke", "ever_told_arthritis",
    "flu_shot_past_year", "ever_had_sigmoidoscopy_colonoscopy",
    "ever_had_mammogram", "ever_had_pap_smear",
    "teeth_cleaned_by_dentist", "last_dentist_visit",
    "high_blood_pressure_flag", "obesity_flag", "overweight_or_obese_flag",
    "smoking_status_recode", "tobacco_use_flag"
]

def safe_select(df, cols):
    return [c for c in cols if c in df.columns]

# --- Build DataFrames (prepend geo once) ---
brfss_socio_core = brfss[safe_select(brfss, geo_cols + socio_vars_core)].copy()
brfss_health_core = brfss[safe_select(brfss, geo_cols + health_vars_core)].copy()
brfss_socio_exp  = brfss[safe_select(brfss, geo_cols + socio_vars_expanded)].copy()
brfss_health_exp = brfss[safe_select(brfss, geo_cols + health_vars_expanded)].copy()

In [ ]:
print(brfss_socio_core.head())
print(brfss_health_core.head())
print(brfss_socio_exp.head())
print(brfss_health_exp.head())

In [ ]:
# --- Save outputs to CSVs in processed/ ---
brfss_socio_core.to_csv(os.path.join(OUT_DIR, f"full_brfss_{TEST_YEAR}_socioeconomics_core.csv"), index=False)
brfss_health_core.to_csv(os.path.join(OUT_DIR, f"full_brfss_{TEST_YEAR}_health_core.csv"), index=False)
brfss_socio_exp.to_csv(os.path.join(OUT_DIR, f"full_brfss_{TEST_YEAR}_socioeconomics_expanded.csv"), index=False)
brfss_health_exp.to_csv(os.path.join(OUT_DIR, f"full_brfss_{TEST_YEAR}_health_expanded.csv"), index=False)

print("✔️ Saved 4 BRFSS outputs in:", os.path.abspath(OUT_DIR))
print(f"   - full_brfss_{TEST_YEAR}_socioeconomics_core.csv")
print(f"   - full_brfss_{TEST_YEAR}_health_core.csv")
print(f"   - full_brfss_{TEST_YEAR}_socioeconomics_expanded.csv")
print(f"   - full_brfss_{TEST_YEAR}_health_expanded.csv")

# List the directory contents to confirm files are there
print("\n📂 Files in processed/:", os.listdir(OUT_DIR))

# --- Preview the saved dataframes ---
print("\n🔎 Preview of saved dataframes:")

print(f"\n[{TEST_YEAR}] Socio Core:", brfss_socio_core.shape)
display(brfss_socio_core.head())

print(f"\n[{TEST_YEAR}] Health Core:", brfss_health_core.shape)
display(brfss_health_core.head())

print(f"\n[{TEST_YEAR}] Socio Expanded:", brfss_socio_exp.shape)
display(brfss_socio_exp.head())

print(f"\n[{TEST_YEAR}] Health Expanded:", brfss_health_exp.shape)
display(brfss_health_exp.head())